In [ ]:
#!pip install tweet-preprocessor

In [12]:
import preprocessor as p
import pandas as pd
import json
import re

In [11]:
pd.options.display.max_colwidth = 75

In [410]:
pwd

'/home/jovyan/capstone-35/gulf_twitter_pickled'

In [1]:
cd ../gulf_twitter_raw/gulf_timelines/

/home/jovyan/capstone-35/gulf_twitter_raw/gulf_timelines


In [2]:
#cd ../gulf_twitter_raw

In [3]:
ls

user_timeline_fhth2020.jsonl


### Turn query documents (tweets) into generators

In [5]:
def load_tweets(file):
    with open(file, 'r') as f:
        tweets = (json.loads(line) for line in f.readlines())
    return tweets

In [6]:
query_gen = load_tweets('user_timeline_fhth2020.jsonl')

### Turn query generators into dataframes

In [7]:
def tweet_to_df(tweets):
    data = {'text': [], 'screen_name': [], 'geo': [],
       }
    
    for t in tweets:
    
        data['text'].append(t['text'])
        data['screen_name'].append(t['user']['screen_name'])
        data['geo'].append(t['user']['location'])
    
    return pd.DataFrame(data)

In [13]:
query_df = tweet_to_df(query_gen)

In [14]:
query_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
geo            0 non-null float64
screen_name    0 non-null float64
text           0 non-null float64
dtypes: float64(3)
memory usage: 76.0 bytes


### Clean tweets (removes emojis, links, special characters, and ASCII chars/digits from tweet text)

Using regular expressions such as `[0-9A-Za-z:&!/._?)؟'+-=—]` can filter out much of the unwanted data and retain Arabic characters but still seems to also retain a more recently released generation of emojis. Try updating tweet-preprocessor to version 0.5.0 or using another tweet processing package. 

Different scenarios will require different regex combinations. For example, the below approach wouldn't be of much use if you're building English dialect classifiers.

In [ ]:
def cleaner(message):
    message = re.sub("([0-9A-Za-z:&!/._?)؟'+-=—])", " ", message)
    return message

In [445]:
cleaned_text = []

for tweet in query_df['text']:
    cleaned_text.append(p.clean(cleaner(tweet)))

query_df['cleaned_text']=cleaned_text

In [446]:
cleaned_name = []

for tweet in query_df['screen_name']:
    cleaned_name.append(p.clean(tweet))

query_df['cleaned_name']=cleaned_name

In [447]:
cleaned_geo = []

for tweet in query_df.geo:
    cleaned_geo.append(p.clean(str(tweet)))

query_df['cleaned_geo']=cleaned_geo


In [448]:
query_df.sample(4)

,geo,screen_name,text,cleaned_text,cleaned_name,cleaned_geo
1431,اعتذر لا ابادل اقل من 10k ❌❌,ME__BROTHER,RT @AFkhamaa: تخفيضات حتى نهايه اليوم \nمفرش شتوي ثقيل وراقي جدا \nيتكو...,تخفيضات حتى نهايه اليوم مفرش شتوي ثقيل وراقي جدا يتكون من قطع لحاف مطرز...,ME__BROTHER,اعتذر لا ابادل اقل من k
572,UG,iii1la,RT @wexlo: بصير ثقيل وبرد متأخر يحسبوني ميت عليهم\n12:00 : الوو\n12:02 ...,بصير ثقيل وبرد متأخر يحسبوني ميت عليهم الوو عيونه,iii1la,UG
1134,الرياض,AMERTA_ANA,RT @a2e355d: ثقيل شعور الشوق ويهلك.. من ثقله كنه حجر يكتم أنفاسي.. 💜,ثقيل شعور الشوق ويهلك من ثقله كنه حجر يكتم أنفاسي,AMERTA_ANA,الرياض
2015,AL DABBOUS KINGDOM FAHAHEEL,akd_boshafi,ما يروح إلا الغبار مع مهب الريح \nولا الذهب ثقيل مقامه,ما يروح إلا الغبار مع مهب الريح ولا الذهب ثقيل مقامه,akd_boshafi,AL DABBOUS KINGDOM FAHAHEEL


### Remove duplicated tweets

In [449]:
query_df['cleaned_text'].duplicated().sample(5)

346      True
1516     True
2049    False
815      True
3223     True
Name: cleaned_text, dtype: bool

In [450]:
query_cleaned = query_df.drop_duplicates(['cleaned_text'], keep=False)
query_cleaned = query_cleaned.drop(['screen_name', 'text', 'geo'], axis=1)

In [451]:
query_cleaned.sample(5)

,cleaned_text,cleaned_name,cleaned_geo
1731,لا مو خفيف ثقيل دم,Ze,None
1205,ولا تجعلني يا الله ثقيل على قلب احد وابعدني عن من يتمنى بٌعدي حتى لو كا...,Shoog81325016,المملكة العربية السعودية
1053,اي خبره و افضليه على الموجودين مهما تواضعت مستوياتهم براعي مشاكل وكروت ...,_ibrahim_otaibi,قلب السعوديه ( نجد )
4203,استسلام شرطي مع رشاش ثقيل وكلاشنكوف للمجاهدين في خاص روزجان,alemara_ar,Afghanistan
4896,الآن يوجد تصدع في قلبي وتخدرفي إدراكي هذا الألم ثقيل جدآ تعال وأنظر لاط...,balshan5959,None


In [452]:
query_cleaned.fillna('None')

,cleaned_text,cleaned_name,cleaned_geo
14,تياقو نيفيز قال اول انا ماراح ارجع للهلال برونو صح لعيب لكنه ثقيل فابيو...,ii_as18,None
18,صباح الخير لكل الذين يعلمون أن الصبر ثقيل وأن الانتظار مرير وأن الكلام ...,A___aisha1,None
23,الحزن عبء ثقيل جدا,Alnshadii,None
24,ارحمنا يا الله لأننا لاندري ، وارحمنا مرة أخرى لأننا حين ندري لا نتمكن ...,al3nziwafa,نصفي دُفن مع أبي.
47,طبعي ثقيل واعشق الثقل والزين واحب اوصل شيء صعبٍ وصوله,m7aamadd,"Manchester, Kuwait"
66,ارحمنا يا الله لأننا لاندري، وارحمنا مرة أخرى لأننا حين ندري لا نتمكن، ...,alkhelagitoma,"جدة, مكة المكرمة"
74,ü ı وهل بعد حديثك يااسيدي سأسعدبحديث غيره حديثنا الجميل بكأس شاي ثقيل و...,Q860Gala,دولة الكويت 🇰🇼
77,الهادي شفت سنابات زوجها وحامت كبدي ثقيل دم مدري كيف متحملته وكل شوي انا...,Sarah_MH6,None
81,تكون السعوديه في اهل الجنوب ليل الشتاء مايشبه الا شعوري بارد ، ثقيل وكل...,S7,يارب انك ترحم أمي️
85,ليا مرني اسمه افز وانا اللي طبعي ثقيل يابري حالي حبيته بكل ماانخلقت من ...,fayez_585,None


In [453]:
query_cleaned.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 912 entries, 14 to 6126
Data columns (total 3 columns):
cleaned_text    912 non-null object
cleaned_name    912 non-null object
cleaned_geo     912 non-null object
dtypes: object(3)
memory usage: 28.5+ KB


### Consolidate different geo tags

In [454]:
locations = {'Saudi Arabia': ['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia', 'الخبر, المملكة العربية السعودي', 'السعودية', 'القصيم', 'تبوك, المملكة العربية السعودية'],
             'Kuwait': ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait', 'دولة الكويت-الفيحاء', 'Al Adailiya, Kuwait'],
             'Egypt': ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egypt', 'Alexandria,Egypt', 'ELsharkia , Egypt', 'شمال سيناء, مصر', 'كفر الشيخ, مصر', 'Port Said, Egypt', 'Beni Suef, Egypt', 'القليوبية, مصر', 'بنها', 'Aswan', 'Beni Suef, Egypt', 'Zagazig,Egypt', 'Ismalia, Egypt', 'Kafr El Shikh, Egypt', 'القاهرة, مصر', 'البحيرة, مصر', 'live in q8 .. bas masrya', 'القاهرة, مصر', 'Domiat, Egypt', 'Alexandria, Egypt', 'Cairo, Egypt', 'El Menia', 'أسكندرية', 'Alexandria', 'tanta', 'giza,egypt', 'Giza Governorate, Egypt', 'Mansoura', 'Mansura,Egypt','المنوفية, مصر', 'مصر الاسكندرية','El Sharkia, Egypt', 'El Daqahlia, Egypt', 'Cairo -Egypt', 'Cairo , Egypt', 'Aswan, Egypt', 'Mansurah, Egypt', 'الأسكندرية, مصر', 'alexandria', 'الأقصر, مصر', 'Cairo', 'El Gharbia, Egypt', 'egypt', 'Mansoura,Egypt', 'Beni Suef, Egypt', 'Mansoura', 'El Daqahlia, Egypt', 'El Monofia, Egypt'],
             'UAE': ['United Arab Emirates', 'Dubai, United Arab Emirates', 'Dubai']
            }

In [455]:
'MAKKAH' in list(locations.values())[0]

True

In [456]:
'MAKKAH' in list(locations.values())[1]

False

In [457]:
locations.values()

dict_values([['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia', 'الخبر, المملكة العربية السعودي', 'السعودية', 'القصيم', 'تبوك, المملكة العربية السعودية'], ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait', 'دولة الكويت-الفيحاء', 'Al Adailiya, Kuwait'], ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elm

In [458]:
for word in query_cleaned['cleaned_geo'].str.split('_'):
    if word[0] in list(locations.values())[0]:
        query_cleaned['cleaned_geo'].replace(word, 'Saudi Arabia', inplace=True)
    elif word[0] in list(locations.values())[1]:
            query_cleaned['cleaned_geo'].replace(word, 'Kuwait', inplace=True)
    elif word[0] in list(locations.values())[2]:
            query_cleaned['cleaned_geo'].replace(word, 'Egypt', inplace=True)
    else:
        if word[0] in list(locations.values())[3]:
            query_cleaned['cleaned_geo'].replace(word, 'UAE', inplace=True)

In [463]:
query_cleaned.sample(5)

,cleaned_text,cleaned_name,cleaned_geo
5456,"""من الأشياء المكروهه ان اكون ثقيل عليك ولم تخبرني بذلك""",ma0997,أدخل الرابط واشبع فضولك
4369,بس ترا ماراح تستمر هالتشكيله اسينسو بيرجع دكه وايسكو بيرجع اساسي وبيرجع...,F_A__RMA,Kuwait
4659,في ناس يسوو انفولو مع كل تغريدة اكتبها ضد العنصرية او ضد التحرش او او ل...,fish_fash,Saudi Arabia
6106,بدر طارق ما يلعب اساسي العازمي كبير محمد الفهد ثقيل الرشيدي ما اعرفه,daif1975,بومحمد
835,فراش ثقيل ، طعام شهي ، شراب دافيء ، نوم هاديء ، أمن وأمان واطمئنان، فـ ...,mostafa_safan,بلاد الله خلق الله


### Pickle to conduct EDA in seperate notebook

In [464]:
pwd

'/home/jovyan/capstone-35/gulf_twitter_raw'

In [466]:
cd ../gulf_twitter_pickled/

/home/jovyan/capstone-35/gulf_twitter_pickled


In [467]:
query_cleaned.to_pickle('../gulf_twitter_pickled/stream_ثقيل.p')